## Quantum Measurement Classification with Mixed States on Ten Classes MNIST with quantum-enhanced Fourier features

Diego Useche

## GPU

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Libraries

In [ ]:
!pip install tensorcircuit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 10.8 MB/s eta 0:00:00


In [ ]:
from functools import partial
import numpy as np
import tensorflow as tf
import tensorcircuit as tc
from tensorcircuit import keras
import math


In [ ]:
tc.set_backend("tensorflow")
tc.set_dtype("complex128")

('complex128', 'float64')

In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score

print(tf.__version__)
print(tf.config.list_logical_devices())

2.17.0
[LogicalDevice(name='/device:CPU:0', device_type='CPU')]


## Utils functions

In [ ]:
# this function takes the number of classes and of qubits of the qmc pure, and extract the indices
# of the bit strings that correpond to the classes prediction
## Example, qmc prediction bit string ["00", "01", "10", "11"]
## the classes prediction is encoded in ["00", "10"], then it returns [0, 2]

def _indices_qubits_classes(num_qubits_param, num_classes_param):
  num_qubits_classes_temp = int(np.ceil(np.log2(num_classes_param)))
  a = [np.binary_repr(i, num_qubits_param) for i in range(2**num_qubits_param)]
  b = [(np.binary_repr(i, num_qubits_classes_temp) + "0"*(num_qubits_param - num_qubits_classes_temp)) for i in range(num_classes_param)]
  indices_temp = []
  for i in range(len(a)):
    if a[i] in b:
      indices_temp.append(i)

  return indices_temp

_indices_qubits_classes(4, 4)

[0, 4, 8, 12]

## MNIST Data Set

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train, X_test = X_train[..., np.newaxis]/255.0, X_test[..., np.newaxis]/255.0

X_train.shape, X_test.shape, y_train.shape, y_test.shape

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


((60000, 28, 28, 1), (10000, 28, 28, 1), (60000,), (10000,))

In [ ]:
# Select the indices for the binary classification
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
y_train_oh = tf.reshape (tf.keras.backend.one_hot(y_train, 10), (-1,10))
y_test_oh = tf.reshape (tf.keras.backend.one_hot(y_test, 10), (-1,10))

X_train.shape, X_test.shape, y_train_oh.shape, y_test_oh.shape

((60000, 28, 28, 1),
 (10000, 28, 28, 1),
 TensorShape([60000, 10]),
 TensorShape([10000, 10]))

## Model 1: Mixed QMC variational, QEFF, with Le-Net Conv layer with APA, Discriminative then generative learning

In [ ]:
### Quantum variational KDC with QEFF

import tensorcircuit as tc
from tensorcircuit import keras
import tensorflow as tf

from functools import partial
import numpy as np
import math as m
from scipy.stats import entropy, spearmanr

tc.set_backend("tensorflow")
tc.set_dtype("complex128")

pi = tf.constant(m.pi)
class VQKDC_MIXED_QEFF_LENET:
    r"""
    Defines the ready-to-use Quantum measurement classification (QMC) model implemented
    in TensorCircuit using the TensorFlow/Keras API. Any additional argument in the methods has to be Keras-compliant.

    Args:
        auto_compile: A boolean to autocompile the model using default settings. (Default True).
        var_pure_state_size:
        gamma:

    Returns:
        An instantiated model ready to train with ad-hoc data.

    """
    def __init__(self, n_qeff_qubits, n_ancilla_qubits, num_classes_qubits, num_classes_param, dim_lenet_out_param, input_dim_param, gamma, n_training_data,  reduction = "none", training_type = "generative", batch_size = 16, learning_rate = 0.0005, random_state = 15, auto_compile=True):

        self.circuit = None
        self.gamma = gamma
        self.num_classes = num_classes_param
        self.num_classes_qubits = num_classes_qubits
        self.n_qeff_qubits = n_qeff_qubits
        self.n_ancilla_qubits = n_ancilla_qubits
        self.n_total_qubits_temp = self.num_classes_qubits + self.n_qeff_qubits + self.n_ancilla_qubits
        self.num_ffs = 2**self.n_qeff_qubits
        self.n_training_data = n_training_data
        self.dim_lenet_out = dim_lenet_out_param
        self.input_dim = input_dim_param
        self.var_pure_state_parameters_size = 2*(2**self.n_total_qubits_temp - 1)
        self.reduction  = reduction
        self.training_type = training_type
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.qeff_weights = tf.random.normal((self.dim_lenet_out, int(self.num_ffs*1-1)), mean = 0.0, stddev = 2.0/np.sqrt(self.num_ffs - 1), dtype=tf.dtypes.float64, seed = random_state)

        layer = keras.QuantumLayer(
            partial(self.layer),
            [(self.var_pure_state_parameters_size,)]
            )

        ## build the Let-net model
        self.model = Sequential()
        self.model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=(5, 5), padding="same", activation='relu', input_shape=(self.input_dim, self.input_dim, 1)))
        self.model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
        self.model.add(tf.keras.layers.Conv2D(filters=50, kernel_size=(5, 5), padding="same", activation='relu'))
        self.model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(units=84, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=self.dim_lenet_out, activation = None)) # original "softmax"

        # add the quantum layer

        self.model.add(layer)
        print(self.model.summary())

        if auto_compile:
            self.compile()

    def layer(
            self,
            x_sample_param,
            var_pure_state_param,
        ):
        r"""
        Defines a Density Matrix Kernel Density Estimation quantum layer for learning with fixed qaff (Meaning of qaff?). (This function was originally named dmkde_mixed_variational_density_estimation_fixed_qaff)

        Args:
            U_dagger:
            var_pure_state_param:

        Returns:
            The probabilities of :math:`|k\rangle`, `|1\rangle`, ..., `|k\rangle` state for kernel density classification of the classes.
        """

        ### indices pure state
        index_it = iter(np.arange(len(var_pure_state_param)))

        ### indices qeff
        index_iter_qeff = iter(np.arange(self.qeff_weights.shape[1]))

        ### indices classes, of ms
        n_qubits_classes_qeff_temp = self.num_classes_qubits + self.n_qeff_qubits
        index_qubit_states = _indices_qubits_classes(n_qubits_classes_qeff_temp, self.num_classes) # extract indices of the bit string of classes


        # Instantiate a circuit with the calculated number of qubits.
        self.circuit = tc.Circuit(self.n_total_qubits_temp)

        def circuit_base_ry_n(qc_param, num_qubits_param, target_qubit_param):
            if num_qubits_param == 1:
                qc_param.ry(0, theta = var_pure_state_param[next(index_it)])
            elif num_qubits_param == 2:
                qc_param.ry(target_qubit_param, theta=var_pure_state_param[next(index_it)])
                qc_param.cnot(0, target_qubit_param)
                qc_param.ry(target_qubit_param, theta=var_pure_state_param[next(index_it)])
                return
            else:
                circuit_base_ry_n(qc_param, num_qubits_param-1, target_qubit_param)
                qc_param.cnot(num_qubits_param-2, target_qubit_param)
                circuit_base_ry_n(qc_param, num_qubits_param-1, target_qubit_param)
                target_qubit_param -= 1

        def circuit_base_rz_n(qc_param, num_qubits_param, target_qubit_param):
            if num_qubits_param == 1:
                qc_param.rz(0, theta = var_pure_state_param[next(index_it)])
            elif num_qubits_param == 2:
                qc_param.rz(target_qubit_param, theta=var_pure_state_param[next(index_it)])
                qc_param.cnot(0, target_qubit_param)
                qc_param.rz(target_qubit_param, theta=var_pure_state_param[next(index_it)])
                return
            else:
                circuit_base_rz_n(qc_param, num_qubits_param-1, target_qubit_param)
                qc_param.cnot(num_qubits_param-2, target_qubit_param)
                circuit_base_rz_n(qc_param, num_qubits_param-1, target_qubit_param)
                target_qubit_param -= 1

        # Learning pure state
        for i in range(1, self.n_total_qubits_temp+1):
            circuit_base_ry_n(self.circuit, i, i-1)

        # Learning pure state complex phase
        for j in range(1, self.n_total_qubits_temp+1):
            circuit_base_rz_n(self.circuit, j, j-1)

        # Value to predict

        x_sample_temp = tf.expand_dims(x_sample_param, axis=0)
        phases_temp = (tf.cast(tf.sqrt(self.gamma), tf.float64)*tf.linalg.matmul(tf.cast(x_sample_temp, tf.float64), self.qeff_weights))[0]
        init_qubit_qeff_temp = self.num_classes_qubits # qubit at which the qaff mapping starts it starts after the qubits of the classes

        def circuit_base_rz_qeff_n(qc_param, num_qubits_param, target_qubit_param, init_qubit_param):
          if num_qubits_param == 1:
            qc_param.rz(init_qubit_param, theta = phases_temp[next(index_iter_qeff)] )
          elif num_qubits_param == 2:
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[next(index_iter_qeff)])
            qc_param.cnot(init_qubit_param, target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[next(index_iter_qeff)])
            return
          else:
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            qc_param.cnot(num_qubits_param-2 + init_qubit_param, target_qubit_param + init_qubit_param)
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            target_qubit_param -= 1

        # Applying the QEFF feature map

        for i in reversed(range(1, self.n_qeff_qubits + 1)):
          circuit_base_rz_qeff_n(self.circuit, i, i - 1, init_qubit_qeff_temp)

        for i in range(init_qubit_qeff_temp, init_qubit_qeff_temp + self.n_qeff_qubits):
          self.circuit.H(i)

        # Trace out ancilla qubits, find probability of [000] state for density estimation
        measurement_state = tc.quantum.reduced_density_matrix(
                        self.circuit.state(),
                        cut=[m for m in range(n_qubits_classes_qeff_temp, self.n_total_qubits_temp)])
        measurements_results = tc.backend.real(tf.stack([measurement_state[index_qubit_states[i], index_qubit_states[i]] for i in range(self.num_classes)]))
        if self.training_type == "discriminative":
          measurements_results = measurements_results / tf.reduce_sum(measurements_results, axis = -1)
        print(self.training_type)
        return measurements_results

    def custom_categorical_crossentropy(self, y_true, y_pred):
      ## code generated with the aid of chat gpt
      """
      Compute the categorical cross-entropy loss with mean reduction.

      Args:
      y_true: Tensor of true labels, shape (batch_size, num_classes).
      y_pred: Tensor of predicted probabilities, shape (batch_size, num_classes).

      Returns:
      Scalar tensor representing the mean loss over the batch.
      """
      # Ensure predictions are clipped to avoid log(0)
      epsilon_two = 1e-7  # small constant to avoid division by zero
      y_pred = tf.clip_by_value(y_pred, epsilon_two, np.inf)  # clip values to avoid log(0) originaly 1.0 - epsilon

      # Compute the categorical cross-entropy loss for each sample
      loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)

      if self.reduction == "none":
        return loss
      elif self.reduction == "mean":
        # Compute the mean loss over the batch
        mean_loss = tf.reduce_mean(loss)
        return mean_loss
      elif self.reduction == "sum":
        # Compute the sum loss over the batch
        sum_loss = tf.reduce_sum(loss)
        return sum_loss
      else:
        return loss

    def compile(
            self,
            optimizer=tf.keras.optimizers.Adam,
            **kwargs):
        r"""
        Method to compile the model.

        Args:
            optimizer:
            **kwargs: Any additional argument.

        Returns:
            None.
        """
        self.model.compile(
            loss = self.custom_categorical_crossentropy,
            optimizer=optimizer(self.learning_rate),
            metrics=["accuracy"],
            **kwargs
        )

    def fit(self, x_train, y_train, batch_size=16, epochs = 30, **kwargs):
        r"""
        Method to fit (train) the model using the ad-hoc dataset.

        Args:
            x_train:
            y_train:
            batch_size:
            epochs:
            **kwargs: Any additional argument.

        Returns:
            None.
        """

        self.model.fit(x_train, y_train, batch_size = self.batch_size, epochs = epochs, **kwargs)

    def predict(self, x_test):
      r"""
      Method to make predictions with the trained model.

      Args:
          x_test:

      Returns:
          The predictions of the conditional density estimation of the input data.
      """
      return (tf.experimental.numpy.power((self.gamma/(pi)), self.dim_lenet_out/2.)*\
          self.model.predict(x_test)).numpy()

    def extract_lenet_features(self, x_test):
      r"""
      Method to extract the features of the Lenet.

      Args:
          x_test:

      Returns:
          Features of the Lenet.
      """
      extract = Model(self.model.inputs, self.model.layers[-2].output)
      new_model = Sequential()
      new_model.add(extract)
      return new_model.predict(x_test)

    def freeze_lenet(self):
      r"""
      Method to frezze the layers of the Lenet.

      Args:
          self:

      Returns:
          None.
      """
      for layer in self.model.layers[:-1]:
          layer.trainable = False

    def set_training_type(self, new_training_type):
      r"""
      Method to change the training type of the method during training.

      Args:
          new_training_strategy:

      Returns:
          None.
      """
      self.training_type = new_training_type

In [ ]:
## training the quantum circuit
# Define a LeNet CNN feature extraction model
input_shape = X_train.shape[1:]
INPUT_DIM = input_shape[0]
NUM_QUBITS_FFS = 4 ## originally N_QEFFS = 16
NUM_CLASSES = 10
NUM_CLASSES_QUBITS = 4
DIM_LENET_OUT = 30 # originally 16
GAMMA = 2**(-2) ## originally 2**(0)
EPOCHS = 5
N_TRAINING_DATA = X_train.shape[0]
BATCH_SIZE = 16
LEARNING_RATE = 0.0005 ## originally 0.0005
RANDOM_STATE_QEFF = 123
NUM_ANCILLA_QUBITS = 1
NUM_TOTAL_QUBITS = NUM_QUBITS_FFS + NUM_ANCILLA_QUBITS + NUM_CLASSES_QUBITS
TRAINING_TYPE = "generative"
REDUCTION = "none"

In [ ]:
## this code creates a discriminative model
vqkdc = VQKDC_MIXED_QEFF_LENET(n_qeff_qubits = NUM_QUBITS_FFS, n_ancilla_qubits =  NUM_ANCILLA_QUBITS, num_classes_qubits = NUM_CLASSES_QUBITS, num_classes_param = NUM_CLASSES, dim_lenet_out_param = DIM_LENET_OUT, input_dim_param = INPUT_DIM, gamma=GAMMA, n_training_data = N_TRAINING_DATA,  reduction = REDUCTION, training_type = TRAINING_TYPE, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, random_state = RANDOM_STATE_QEFF, auto_compile = False)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
vqkdc.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
vqkdc.fit(X_train, y_train_oh, epochs = EPOCHS)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


generative


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 20)          │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 14, 14, 20)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 14, 14, 50)          │          25,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_1                  │ (None, 7, 7, 50)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2450)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 84)                  │         205,884 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ quantum_layer (QuantumLayer)         │ (None, 10)                  │           1,022 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 235,026 (922.06 KB)

 Trainable params: 235,026 (922.06 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
generative
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1447s 225ms/step - accuracy: 0.2856 - loss: 3.1587
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 900s 240ms/step - accuracy: 0.6092 - loss: 0.9907
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 996s 260ms/step - accuracy: 0.8978 - loss: 0.3020
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 926s 245ms/step - accuracy: 0.9812 - loss: 0.1097
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 888s 236ms/step - accuracy: 0.9860 - loss: 0.0682


In [ ]:
# this code frezzes the weights of the Le net layer, and then sets the model to be trained in a generative way
for layer in vqkdc.model.layers[:-1]:
    layer.trainable = False
vqkdc.compile()
vqkdc.fit(X_train, y_train_oh, epochs = EPOCHS)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 951s 227ms/step - accuracy: 0.9178 - loss: 2.9227
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 829s 218ms/step - accuracy: 0.9845 - loss: 2.3402
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 827s 220ms/step - accuracy: 0.9840 - loss: 2.3409
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 917s 235ms/step - accuracy: 0.9841 - loss: 2.3403
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 882s 235ms/step - accuracy: 0.9844 - loss: 2.3400


In [ ]:
y_pred = vqkdc.predict(X_test)

accuracy_score(y_test, np.argmax(y_pred, axis=1))

generative
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/stepgenerative
313/313 ━━━━━━━━━━━━━━━━━━━━ 940s 2s/step


0.9821

## Model 2: Mixed QMC variational, QEFF, with Le-Net Conv layer with HEA, Discriminative then generative learning

The number of parameters of the Hardware efficient ansatz is given by,


```
HEA_size = n_qubits * (num_layers_hea + 1) * 2
```



In [ ]:
### Quantum variational KDC with QEFF

import tensorcircuit as tc
from tensorcircuit import keras
import tensorflow as tf

from functools import partial
import numpy as np
import math as m
from scipy.stats import entropy, spearmanr

tc.set_backend("tensorflow")
tc.set_dtype("complex128")

pi = tf.constant(m.pi)

class VQKDC_MIXED_QEFF_HEA_LENET:
    r"""
    Defines the ready-to-use Quantum measurement classification (QMC) model implemented
    in TensorCircuit using the TensorFlow/Keras API. Any additional argument in the methods has to be Keras-compliant.

    Args:
        auto_compile: A boolean to autocompile the model using default settings. (Default True).
        var_pure_state_size:
        gamma:

    Returns:
        An instantiated model ready to train with ad-hoc data.

    """
    def __init__(self, n_qeff_qubits, n_ancilla_qubits, num_classes_qubits, num_classes_param, dim_lenet_out_param,  input_dim_param, gamma, n_training_data, reduction = "none", training_type = "generative", num_layers_hea = 3, batch_size = 16, learning_rate = 0.0005, random_state = 15, auto_compile=True):

        self.circuit = None
        self.gamma = gamma
        self.num_layers_hea = num_layers_hea
        self.num_classes = num_classes_param
        self.num_classes_qubits = num_classes_qubits
        self.n_qeff_qubits = n_qeff_qubits
        self.n_ancilla_qubits = n_ancilla_qubits
        self.n_total_qubits_temp = self.num_classes_qubits + self.n_qeff_qubits + self.n_ancilla_qubits
        self.num_ffs = 2**self.n_qeff_qubits
        self.n_training_data = n_training_data
        self.dim_lenet_out = dim_lenet_out_param
        self.input_dim = input_dim_param
        self.var_hea_ansatz_size = int(self.n_total_qubits_temp*(self.num_layers_hea+1)*2)
        self.reduction  = reduction
        self.training_type = training_type
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.qeff_weights = tf.random.normal((self.dim_lenet_out, int(self.num_ffs*1-1)), mean = 0.0, stddev = 2.0/np.sqrt(self.num_ffs - 1), dtype=tf.dtypes.float64, seed = random_state)

        layer = keras.QuantumLayer(
            partial(self.layer),
            [(self.var_hea_ansatz_size,)]
            )

        ## build the Let-net model
        self.model = Sequential()
        self.model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=(5, 5), padding="same", activation='relu', input_shape=(self.input_dim, self.input_dim, 1)))
        self.model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
        self.model.add(tf.keras.layers.Conv2D(filters=50, kernel_size=(5, 5), padding="same", activation='relu'))
        self.model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(units=84, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=self.dim_lenet_out, activation = None)) # original "softmax"

        # add the quantum layer

        self.model.add(layer)
        print(self.model.summary())

        if auto_compile:
            self.compile()

    def layer(
            self,
            x_sample_param,
            var_hea_ansatz_param,
        ):
        r"""
        Defines a Density Matrix Kernel Density Estimation quantum layer for learning with fixed qaff (Meaning of qaff?). (This function was originally named dmkde_mixed_variational_density_estimation_fixed_qaff)

        Args:
            U_dagger:
            var_pure_state_param:

        Returns:
            The probabilities of :math:`|k\rangle`, `|1\rangle`, ..., `|k\rangle` state for kernel density classification of the classes.
        """

        ### indices pure state hea
        index_iter_hea  = iter(np.arange(len(var_hea_ansatz_param)))

        ### indices qeff
        index_iter_qeff = iter(np.arange(self.qeff_weights.shape[1]))

        ### indices classes, of ms
        n_qubits_classes_qeff_temp = self.num_classes_qubits + self.n_qeff_qubits
        index_qubit_states = _indices_qubits_classes(n_qubits_classes_qeff_temp, self.num_classes) # extract indices of the bit string of classes


        # Instantiate a circuit with the calculated number of qubits.
        self.circuit = tc.Circuit(self.n_total_qubits_temp)

        def hea_ansatz(qc_param, num_qubits_param, num_layers_param):
          # encoding
          for i in range (0, num_qubits_param):
            qc_param.ry(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
            qc_param.rz(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
          # layers
          for j in range(num_layers_param):
            for i in range (0, num_qubits_param-1):
              qc_param.CNOT(i, i+1)

            for i in range (0, num_qubits_param):
              qc_param.ry(i, theta= var_hea_ansatz_param[next(index_iter_hea)])
              qc_param.rz(i, theta= var_hea_ansatz_param[next(index_iter_hea)])

        ## learning pure state with HEA
        hea_ansatz(self.circuit, self.n_total_qubits_temp, self.num_layers_hea)

        # Value to predict

        x_sample_temp = tf.expand_dims(x_sample_param, axis=0)
        phases_temp = (tf.cast(tf.sqrt(self.gamma), tf.float64)*tf.linalg.matmul(tf.cast(x_sample_temp, tf.float64), self.qeff_weights))[0]
        init_qubit_qeff_temp = self.num_classes_qubits # qubit at which the qaff mapping starts it starts after the qubits of the classes

        def circuit_base_rz_qeff_n(qc_param, num_qubits_param, target_qubit_param, init_qubit_param):
          if num_qubits_param == 1:
            qc_param.rz(init_qubit_param, theta = phases_temp[next(index_iter_qeff)] )
          elif num_qubits_param == 2:
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[next(index_iter_qeff)])
            qc_param.cnot(init_qubit_param, target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[next(index_iter_qeff)])
            return
          else:
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            qc_param.cnot(num_qubits_param-2 + init_qubit_param, target_qubit_param + init_qubit_param)
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            target_qubit_param -= 1

        # Applying the QEFF feature map

        for i in reversed(range(1, self.n_qeff_qubits + 1)):
          circuit_base_rz_qeff_n(self.circuit, i, i - 1, init_qubit_qeff_temp)

        for i in range(init_qubit_qeff_temp, init_qubit_qeff_temp + self.n_qeff_qubits):
          self.circuit.H(i)

        # Trace out ancilla qubits, find probability of [000] state for density estimation
        measurement_state = tc.quantum.reduced_density_matrix(
                        self.circuit.state(),
                        cut=[m for m in range(n_qubits_classes_qeff_temp, self.n_total_qubits_temp)])
        measurements_results = tc.backend.real(tf.stack([measurement_state[index_qubit_states[i], index_qubit_states[i]] for i in range(self.num_classes)]))
        if self.training_type == "discriminative":
          measurements_results = measurements_results / tf.reduce_sum(measurements_results, axis = -1)
        return measurements_results

    def custom_categorical_crossentropy(self, y_true, y_pred):
      ## code generated with the aid of chat gpt
      """
      Compute the categorical cross-entropy loss with mean reduction.

      Args:
      y_true: Tensor of true labels, shape (batch_size, num_classes).
      y_pred: Tensor of predicted probabilities, shape (batch_size, num_classes).

      Returns:
      Scalar tensor representing the mean loss over the batch.
      """
      # Ensure predictions are clipped to avoid log(0)
      epsilon_two = 1e-7  # small constant to avoid division by zero
      y_pred = tf.clip_by_value(y_pred, epsilon_two, np.inf)  # clip values to avoid log(0) originaly 1.0 - epsilon

      # Compute the categorical cross-entropy loss for each sample
      loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)

      if self.reduction == "none":
        return loss
      elif self.reduction == "mean":
        # Compute the mean loss over the batch
        mean_loss = tf.reduce_mean(loss)
        return mean_loss
      elif self.reduction == "sum":
        # Compute the sum loss over the batch
        sum_loss = tf.reduce_sum(loss)
        return sum_loss
      else:
        return loss

    def compile(
            self,
            optimizer=tf.keras.optimizers.Adam,
            **kwargs):
        r"""
        Method to compile the model.

        Args:
            optimizer:
            **kwargs: Any additional argument.

        Returns:
            None.
        """
        self.model.compile(
            loss = self.custom_categorical_crossentropy,
            optimizer=optimizer(self.learning_rate),
            metrics=["accuracy"],
            **kwargs
        )

    def fit(self, x_train, y_train, batch_size=16, epochs = 30, **kwargs):
        r"""
        Method to fit (train) the model using the ad-hoc dataset.

        Args:
            x_train:
            y_train:
            batch_size:
            epochs:
            **kwargs: Any additional argument.

        Returns:
            None.
        """

        self.model.fit(x_train, y_train, batch_size = self.batch_size, epochs = epochs, **kwargs)

    def predict(self, x_test):
      r"""
      Method to make predictions with the trained model.

      Args:
          x_test:

      Returns:
          The predictions of the conditional density estimation of the input data.
      """
      return (tf.experimental.numpy.power((self.gamma/(pi)), self.dim_lenet_out/2.)*\
          self.model.predict(x_test)).numpy()

    def extract_lenet_features(self, x_test):
      r"""
      Method to extract the features of the Lenet.

      Args:
          x_test:

      Returns:
          Features of the Lenet.
      """
      extract = Model(self.model.inputs, self.model.layers[-2].output)
      new_model = Sequential()
      new_model.add(extract)
      return new_model.predict(x_test)

    def freeze_lenet(self):
      r"""
      Method to frezze the layers of the Lenet.

      Args:
          self:

      Returns:
          None.
      """
      for layer in self.model.layers[:-1]:
          layer.trainable = False

    def set_training_type(self, new_training_type):
      r"""
      Method to change the training type of the method during training.

      Args:
          new_training_strategy:

      Returns:
          None.
      """
      self.training_type = new_training_type

In [ ]:
## training the quantum circuit
# Define a LeNet CNN feature extraction model
input_shape = X_train.shape[1:]
INPUT_DIM = input_shape[0]
NUM_QUBITS_FFS = 4 ## originally N_QEFFS = 16
NUM_CLASSES = 10
NUM_CLASSES_QUBITS = 4
DIM_LENET_OUT = 30 # originally 16
GAMMA = 2**(-2) ## originally 2**(0)
EPOCHS = 5
N_TRAINING_DATA = X_train.shape[0]
BATCH_SIZE = 16
LEARNING_RATE = 0.0005 ## originally 0.0005
RANDOM_STATE_QEFF = 123
NUM_ANCILLA_QUBITS = 1
NUM_TOTAL_QUBITS = NUM_QUBITS_FFS + NUM_ANCILLA_QUBITS + NUM_CLASSES_QUBITS
NUM_LAYERS_HEA = int(np.round(((2**NUM_TOTAL_QUBITS-1)/NUM_TOTAL_QUBITS)-1))
TRAINING_TYPE = "generative"
REDUCTION = "none"

In [ ]:
## this code creates a discriminative model
vqkdc_hea = VQKDC_MIXED_QEFF_HEA_LENET(n_qeff_qubits = NUM_QUBITS_FFS, n_ancilla_qubits =  NUM_ANCILLA_QUBITS, num_classes_qubits = NUM_CLASSES_QUBITS, num_classes_param = NUM_CLASSES, dim_lenet_out_param = DIM_LENET_OUT, input_dim_param = INPUT_DIM, gamma=GAMMA, n_training_data = N_TRAINING_DATA,  reduction = REDUCTION, training_type = TRAINING_TYPE, num_layers_hea = NUM_LAYERS_HEA, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, random_state = RANDOM_STATE_QEFF, auto_compile = False)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
vqkdc_hea.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
vqkdc_hea.fit(X_train, y_train_oh, epochs = EPOCHS)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 20)          │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_2                  │ (None, 14, 14, 20)          │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 14, 14, 50)          │          25,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_3                  │ (None, 7, 7, 50)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 2450)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 84)                  │         205,884 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 30)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ quantum_layer_1 (QuantumLayer)       │ (None, 10)                  │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 235,030 (922.09 KB)

 Trainable params: 235,030 (922.09 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1361s 232ms/step - accuracy: 0.7808 - loss: 0.9387
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 855s 228ms/step - accuracy: 0.9879 - loss: 0.0845
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 869s 230ms/step - accuracy: 0.9919 - loss: 0.0504
Epoch 4/5
 744/3750 ━━━━━━━━━━━━━━━━━━━━ 13:22 267ms/step - accuracy: 0.9948 - loss: 0.0307

In [ ]:
# this code frezzes the weights of the Le net layer, and then sets the model to be trained in a generative way
for layer in vqkdc_hea.model.layers[:-1]:
    layer.trainable = False
vqkdc_hea.compile()
vqkdc_hea.fit(X_train, y_train_oh, epochs = EPOCHS)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 773s 178ms/step - accuracy: 0.9843 - loss: 2.7037
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 679s 178ms/step - accuracy: 0.9878 - loss: 2.3425
Epoch 3/5
 431/3750 ━━━━━━━━━━━━━━━━━━━━ 9:39 175ms/step - accuracy: 0.9877 - loss: 2.3361

KeyboardInterrupt: 

In [ ]:
y_pred = vqkdc_hea.predict(X_test)

accuracy_score(y_test, np.argmax(y_pred, axis=1))

313/313 ━━━━━━━━━━━━━━━━━━━━ 628s 1s/step


0.9864

## Baseline model: Mixed QMC variational, ZZFM, with Le-Net Conv layer with HEA, Discriminative then generative learning, additional linear layer from dim of latent space to dim 4.

The number of parameters of the Hardware efficient ansatz is given by,


```
HEA_size = n_qubits * (num_layers_hea + 1) * 2
```



In [ ]:
### Quantum variational KDC with QEFF

import tensorcircuit as tc
from tensorcircuit import keras
import tensorflow as tf

from functools import partial
import numpy as np
import math as m
from scipy.stats import entropy, spearmanr

tc.set_backend("tensorflow")
tc.set_dtype("complex128")

pi = tf.constant(m.pi)

class VQKDC_MIXED_ZZFM_HEA_LENET:
    r"""
    Defines the ready-to-use Quantum measurement classification (QMC) model implemented
    in TensorCircuit using the TensorFlow/Keras API. Any additional argument in the methods has to be Keras-compliant.

    Args:
        auto_compile: A boolean to autocompile the model using default settings. (Default True).
        var_pure_state_size:
        gamma:

    Returns:
        An instantiated model ready to train with ad-hoc data.

    """
    def __init__(self, n_qeff_qubits, n_ancilla_qubits, num_classes_qubits, num_classes_param, dim_lenet_out_param,  input_dim_param, gamma, n_training_data, num_zzfm_layers, reduction = "none", training_type = "generative", num_layers_hea = 3, batch_size = 16, learning_rate = 0.0005, auto_compile=True):

        self.circuit = None
        self.gamma = gamma
        self.num_layers_hea = num_layers_hea
        self.num_classes = num_classes_param
        self.num_classes_qubits = num_classes_qubits
        self.n_zzfm_qubits = n_qeff_qubits
        self.n_ancilla_qubits = n_ancilla_qubits
        self.n_total_qubits_temp = self.num_classes_qubits + self.n_zzfm_qubits + self.n_ancilla_qubits
        self.n_training_data = n_training_data
        self.dim_lenet_out = dim_lenet_out_param
        self.input_dim = input_dim_param
        self.num_zzfm_layers = num_zzfm_layers
        self.var_hea_ansatz_size = int(self.n_total_qubits_temp*(self.num_layers_hea+1)*2)
        self.reduction  = reduction
        self.training_type = training_type
        self.learning_rate = learning_rate
        self.batch_size = batch_size

        layer = keras.QuantumLayer(
            partial(self.layer),
            [(self.var_hea_ansatz_size,)]
            )

        ## build the Let-net model
        #self.model = Sequential() ## keep for old code
        self.model = Sequential() ## new code
        self.model.add(tf.keras.layers.InputLayer(input_shape=(self.input_dim, self.input_dim, 1))) ## new code
        #self.model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=(5, 5), padding="same", activation='relu', input_shape=(self.input_dim, self.input_dim, 1))) # keep for old code
        self.model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=(5, 5), padding="same", activation='relu')) ## new code
        self.model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
        self.model.add(tf.keras.layers.Conv2D(filters=50, kernel_size=(5, 5), padding="same", activation='relu'))
        self.model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(units=84, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=self.dim_lenet_out, activation = None)) # original "softmax"
        self.model.add(tf.keras.layers.Dense(units=self.n_zzfm_qubits, activation = None))  ## we add this additional layer compared to normal models

        # add the quantum layer

        self.model.add(layer)
        print(self.model.summary())

        if auto_compile:
            self.compile()

    def layer(
            self,
            x_sample_param,
            var_hea_ansatz_param,
        ):
        r"""
        Defines a Density Matrix Kernel Density Estimation quantum layer for learning with fixed qaff (Meaning of qaff?). (This function was originally named dmkde_mixed_variational_density_estimation_fixed_qaff)

        Args:
            U_dagger:
            var_pure_state_param:

        Returns:
            The probabilities of :math:`|k\rangle`, `|1\rangle`, ..., `|k\rangle` state for kernel density classification of the classes.
        """

        ### indices pure state hea
        index_iter_hea  = iter(np.arange(len(var_hea_ansatz_param)))

        ### indices classes, of ms
        n_qubits_classes_zzfm_temp = self.num_classes_qubits + self.n_zzfm_qubits
        index_qubit_states = _indices_qubits_classes(n_qubits_classes_zzfm_temp, self.num_classes) # extract indices of the bit string of classes

        # Instantiate a circuit with the calculated number of qubits.
        self.circuit = tc.Circuit(self.n_total_qubits_temp)

        def hea_ansatz(qc_param, num_qubits_param, num_layers_param):
          # encoding
          for i in range (0, num_qubits_param):
            qc_param.ry(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
            qc_param.rz(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
          # layers
          for j in range(num_layers_param):
            for i in range (0, num_qubits_param-1):
              qc_param.CNOT(i, i+1)

            for i in range (0, num_qubits_param):
              qc_param.ry(i, theta= var_hea_ansatz_param[next(index_iter_hea)])
              qc_param.rz(i, theta= var_hea_ansatz_param[next(index_iter_hea)])

        ## learning pure state with HEA
        hea_ansatz(self.circuit, self.n_total_qubits_temp, self.num_layers_hea)

        # Value to predict zzfm
        x_sample_temp = tf.expand_dims(x_sample_param, axis=0)
        init_qubit_qeff_temp = self.num_classes_qubits # qubit at which the qaff mapping starts it starts after the qubits of the classes

        for i in range(self.num_zzfm_layers):
          for k in reversed(range(self.n_zzfm_qubits)):
            for j in reversed(range(0, k)):
              self.circuit.cnot(init_qubit_qeff_temp + j, init_qubit_qeff_temp + k)
              self.circuit.rz(init_qubit_qeff_temp + k, theta = 2*(math.pi-x_sample_temp[0][j])*(math.pi-x_sample_temp[0][k]))
              self.circuit.cnot(init_qubit_qeff_temp + j, init_qubit_qeff_temp + k)

          for l in range(self.n_zzfm_qubits):
            self.circuit.rz(init_qubit_qeff_temp + l, theta = 2*(x_sample_temp[0][l]))

          for m in range(init_qubit_qeff_temp, init_qubit_qeff_temp + self.n_zzfm_qubits):
            self.circuit.H(m)

        # Trace out ancilla qubits, find probability of [000] state for density estimation
        measurement_state = tc.quantum.reduced_density_matrix(
                        self.circuit.state(),
                        cut=[m for m in range(n_qubits_classes_zzfm_temp, self.n_total_qubits_temp)])
        measurements_results = tc.backend.real(tf.stack([measurement_state[index_qubit_states[i], index_qubit_states[i]] for i in range(self.num_classes)]))
        if self.training_type == "discriminative":
          measurements_results = measurements_results / tf.reduce_sum(measurements_results, axis = -1)
        return measurements_results

    def custom_categorical_crossentropy(self, y_true, y_pred):
      ## code generated with the aid of chat gpt
      """
      Compute the categorical cross-entropy loss with mean reduction.

      Args:
      y_true: Tensor of true labels, shape (batch_size, num_classes).
      y_pred: Tensor of predicted probabilities, shape (batch_size, num_classes).

      Returns:
      Scalar tensor representing the mean loss over the batch.
      """
      # Ensure predictions are clipped to avoid log(0)
      epsilon_two = 1e-7  # small constant to avoid division by zero
      y_pred = tf.clip_by_value(y_pred, epsilon_two, np.inf)  # clip values to avoid log(0) originaly 1.0 - epsilon

      # Compute the categorical cross-entropy loss for each sample
      loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)

      if self.reduction == "none":
        return loss
      elif self.reduction == "mean":
        # Compute the mean loss over the batch
        mean_loss = tf.reduce_mean(loss)
        return mean_loss
      elif self.reduction == "sum":
        # Compute the sum loss over the batch
        sum_loss = tf.reduce_sum(loss)
        return sum_loss
      else:
        return loss

    def compile(
            self,
            optimizer=tf.keras.optimizers.Adam,
            **kwargs):
        r"""
        Method to compile the model.

        Args:
            optimizer:
            **kwargs: Any additional argument.

        Returns:
            None.
        """
        self.model.compile(
            loss = self.custom_categorical_crossentropy,
            optimizer=optimizer(self.learning_rate),
            metrics=["accuracy"],
            **kwargs
        )

    def fit(self, x_train, y_train, batch_size=16, epochs = 30, **kwargs):
        r"""
        Method to fit (train) the model using the ad-hoc dataset.

        Args:
            x_train:
            y_train:
            batch_size:
            epochs:
            **kwargs: Any additional argument.

        Returns:
            None.
        """

        self.model.fit(x_train, y_train, batch_size = self.batch_size, epochs = epochs, **kwargs)

    def predict(self, x_test):
      r"""
      Method to make predictions with the trained model.

      Args:
          x_test:

      Returns:
          The predictions of the conditional density estimation of the input data.
      """
      return (tf.experimental.numpy.power((self.gamma/(pi)), self.dim_lenet_out/2.)*\
          self.model.predict(x_test)).numpy()

    def extract_lenet_features(self, x_test):
      r"""
      Method to extract the features of the Lenet.

      Args:
          x_test:

      Returns:
          Features of the Lenet.
      """
      extract = Model(self.model.inputs, self.model.layers[-2].output)
      new_model = Sequential()
      new_model.add(extract)
      return new_model.predict(x_test)

    def freeze_lenet(self):
      r"""
      Method to frezze the layers of the Lenet.

      Args:
          self:

      Returns:
          None.
      """
      for layer in self.model.layers[:-1]:
          layer.trainable = False

    def set_training_type(self, new_training_type):
      r"""
      Method to change the training type of the method during training.

      Args:
          new_training_strategy:

      Returns:
          None.
      """
      self.training_type = new_training_type

In [ ]:
class VQKDC_MIXED_ZZFM_HEA:
    r"""
    Defines the ready-to-use Quantum measurement classification (QMC) model implemented
    in TensorCircuit using the TensorFlow/Keras API. Any additional argument in the methods has to be Keras-compliant.

    Args:
        auto_compile: A boolean to autocompile the model using default settings. (Default True).
        var_pure_state_size:
        gamma:

    Returns:
        An instantiated model ready to train with ad-hoc data.

    """
    def __init__(self, dim_x_param, n_zzfm_qubits, n_ancilla_qubits, num_classes_qubits, num_classes_param, n_training_data, num_zzfm_layers, reduction = "none", training_type = "generative", num_layers_hea = 3, batch_size = 16, learning_rate = 0.0005, auto_compile=True):

        self.circuit = None
        self.dim_x = dim_x_param
        self.num_layers_hea = num_layers_hea
        self.num_classes = num_classes_param
        self.num_classes_qubits = num_classes_qubits
        self.n_zzfm_qubits = n_zzfm_qubits
        self.n_ancilla_qubits = n_ancilla_qubits
        self.n_total_qubits_temp = self.num_classes_qubits + self.n_zzfm_qubits + self.n_ancilla_qubits
        self.n_training_data = n_training_data
        self.num_zzfm_layers = num_zzfm_layers
        self.var_hea_ansatz_size = int(self.n_total_qubits_temp*(self.num_layers_hea+1)*2)
        self.reduction  = reduction
        self.training_type = training_type
        self.learning_rate = learning_rate
        self.batch_size = batch_size

        layer = keras.QuantumLayer(
            partial(self.layer),
            [(self.var_hea_ansatz_size,)]
            )

        self.model = tf.keras.Sequential([layer])

        if auto_compile:
            self.compile()

    def layer(
            self,
            x_sample_param,
            var_hea_ansatz_param,
        ):
        r"""
        Defines a Density Matrix Kernel Density Estimation quantum layer for learning with fixed qaff (Meaning of qaff?). (This function was originally named dmkde_mixed_variational_density_estimation_fixed_qaff)

        Args:
            U_dagger:
            var_pure_state_param:

        Returns:
            The probabilities of :math:`|k\rangle`, `|1\rangle`, ..., `|k\rangle` state for kernel density classification of the classes.
        """

        ### indices pure state hea
        index_iter_hea  = iter(np.arange(len(var_hea_ansatz_param)))

        ### indices classes, of ms
        n_qubits_classes_zzfm_temp = self.num_classes_qubits + self.n_zzfm_qubits
        index_qubit_states = _indices_qubits_classes(n_qubits_classes_zzfm_temp, self.num_classes) # extract indices of the bit string of classes


        # Instantiate a circuit with the calculated number of qubits.
        self.circuit = tc.Circuit(self.n_total_qubits_temp)

        def hea_ansatz(qc_param, num_qubits_param, num_layers_param):
          # encoding
          for i in range (0, num_qubits_param):
            qc_param.ry(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
            qc_param.rz(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
          # layers
          for j in range(num_layers_param):
            for i in range (0, num_qubits_param-1):
              qc_param.CNOT(i, i+1)

            for i in range (0, num_qubits_param):
              qc_param.ry(i, theta= var_hea_ansatz_param[next(index_iter_hea)])
              qc_param.rz(i, theta= var_hea_ansatz_param[next(index_iter_hea)])

        ## learning pure state with HEA
        hea_ansatz(self.circuit, self.n_total_qubits_temp, self.num_layers_hea)

        # Value to predict zzfm
        x_sample_temp = tf.expand_dims(x_sample_param, axis=0)
        init_qubit_qeff_temp = self.num_classes_qubits # qubit at which the qaff mapping starts it starts after the qubits of the classes

        for i in range(self.num_zzfm_layers):
          for k in reversed(range(self.n_zzfm_qubits)):
            for j in reversed(range(0, k)):
              self.circuit.cnot(init_qubit_qeff_temp + j, init_qubit_qeff_temp + k)
              self.circuit.rz(init_qubit_qeff_temp + k, theta = 2*(math.pi-x_sample_temp[0][j])*(math.pi-x_sample_temp[0][k]))
              self.circuit.cnot(init_qubit_qeff_temp + j, init_qubit_qeff_temp + k)

          for l in range(self.n_zzfm_qubits):
            self.circuit.rz(init_qubit_qeff_temp + l, theta = 2*(x_sample_temp[0][l]))

          for m in range(init_qubit_qeff_temp, init_qubit_qeff_temp + self.n_zzfm_qubits):
            self.circuit.H(m)

        # Trace out ancilla qubits, find probability of [000] state for density estimation
        measurement_state = tc.quantum.reduced_density_matrix(
                        self.circuit.state(),
                        cut=[m for m in range(n_qubits_classes_zzfm_temp, self.n_total_qubits_temp)])
        measurements_results = tc.backend.real(tf.stack([measurement_state[index_qubit_states[i], index_qubit_states[i]] for i in range(self.num_classes)]))
        if self.training_type == "discriminative":
          measurements_results = measurements_results / tf.reduce_sum(measurements_results, axis = -1)
        return measurements_results

    def custom_categorical_crossentropy(self, y_true, y_pred):
      ## code generated with the aid of chat gpt
      """
      Compute the categorical cross-entropy loss with mean reduction.

      Args:
      y_true: Tensor of true labels, shape (batch_size, num_classes).
      y_pred: Tensor of predicted probabilities, shape (batch_size, num_classes).

      Returns:
      Scalar tensor representing the mean loss over the batch.
      """
      # Ensure predictions are clipped to avoid log(0)
      epsilon_two = 1e-7  # small constant to avoid division by zero
      y_pred = tf.clip_by_value(y_pred, epsilon_two, np.inf)  # clip values to avoid log(0) originaly 1.0 - epsilon

      # Compute the categorical cross-entropy loss for each sample
      loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)

      if self.reduction == "none":
        return loss
      elif self.reduction == "mean":
        # Compute the mean loss over the batch
        mean_loss = tf.reduce_mean(loss)
        return mean_loss
      elif self.reduction == "sum":
        # Compute the sum loss over the batch
        sum_loss = tf.reduce_sum(loss)
        return sum_loss
      else:
        return loss

    def compile(
            self,
            optimizer=tf.keras.optimizers.Adam,
            **kwargs):
        r"""
        Method to compile the model.

        Args:
            optimizer:
            **kwargs: Any additional argument.

        Returns:
            None.
        """
        self.model.compile(
            loss = self.custom_categorical_crossentropy,
            optimizer=optimizer(self.learning_rate),
            metrics=["accuracy"],
            **kwargs
        )

    def fit(self, x_train, y_train, batch_size=16, epochs = 30, **kwargs):
        r"""
        Method to fit (train) the model using the ad-hoc dataset.

        Args:
            x_train:
            y_train:
            batch_size:
            epochs:
            **kwargs: Any additional argument.

        Returns:
            None.
        """

        self.model.fit(x_train, y_train, batch_size = self.batch_size, epochs = epochs, **kwargs)

    def predict(self, x_test):
      r"""
      Method to make predictions with the trained model.

      Args:
          x_test:

      Returns:
          The predictions of the conditional density estimation of the input data.
      """
      return self.model.predict(x_test)

In [ ]:
## training the quantum circuit
# Define a LeNet CNN feature extraction model
input_shape = X_train.shape[1:]
INPUT_DIM = input_shape[0]
NUM_QUBITS_FFS = 4 ## originally N_QEFFS = 16
NUM_CLASSES = 10
NUM_CLASSES_QUBITS = 4
DIM_LENET_OUT = 30 # originally 16
GAMMA = 2**(-2) ## originally 2**(0)
EPOCHS = 4
N_TRAINING_DATA = X_train.shape[0]
BATCH_SIZE = 16
NUM_ZZFM_LAYERS = 2 ## set 2 for final experiments
LEARNING_RATE = 0.0005 ## originally 0.0005
RANDOM_STATE_QEFF = 123
NUM_ANCILLA_QUBITS = 1
NUM_TOTAL_QUBITS = NUM_QUBITS_FFS + NUM_ANCILLA_QUBITS + NUM_CLASSES_QUBITS
NUM_LAYERS_HEA = int(np.round(((2**NUM_TOTAL_QUBITS-1)/NUM_TOTAL_QUBITS)-1))
TRAINING_TYPE = "generative"
REDUCTION = "none"

In [ ]:
## this code creates a discriminative model
vqkdc_zzfm_hea_lenet = VQKDC_MIXED_ZZFM_HEA_LENET(n_qeff_qubits = NUM_QUBITS_FFS, n_ancilla_qubits =  NUM_ANCILLA_QUBITS, num_classes_qubits = NUM_CLASSES_QUBITS, num_classes_param = NUM_CLASSES, dim_lenet_out_param = DIM_LENET_OUT, input_dim_param = INPUT_DIM, gamma=GAMMA, n_training_data = N_TRAINING_DATA, num_zzfm_layers = NUM_ZZFM_LAYERS, reduction = REDUCTION, training_type = TRAINING_TYPE, num_layers_hea = NUM_LAYERS_HEA, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, auto_compile = False)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
vqkdc_zzfm_hea_lenet.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
vqkdc_zzfm_hea_lenet.fit(X_train, y_train_oh, epochs = EPOCHS)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 20)          │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 14, 14, 20)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 14, 14, 50)          │          25,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_1                  │ (None, 7, 7, 50)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2450)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 84)                  │         205,884 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │             124 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ quantum_layer (QuantumLayer)         │ (None, 10)                  │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 235,154 (922.58 KB)

 Trainable params: 235,154 (922.58 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/4
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1202s 211ms/step - accuracy: 0.7149 - loss: 1.1444
Epoch 2/4
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 790s 211ms/step - accuracy: 0.9817 - loss: 0.1589
Epoch 3/4
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 793s 212ms/step - accuracy: 0.9873 - loss: 0.0982
Epoch 4/4
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 802s 212ms/step - accuracy: 0.9908 - loss: 0.0815


In [ ]:
# Create a model only for feature extraction with intermediate sigmoid activation
extract = tf.keras.Model(vqkdc_zzfm_hea_lenet.model.inputs, vqkdc_zzfm_hea_lenet.model.layers[-2].output)
print(extract.summary())

X_train_feats = extract.predict(X_train)
X_test_feats = extract.predict(X_test)

X_train_feats.shape, X_test_feats.shape

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 28, 28, 20)          │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 14, 14, 20)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 14, 14, 50)          │          25,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_1                  │ (None, 7, 7, 50)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2450)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 84)                  │         205,884 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │             124 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 234,128 (914.56 KB)

 Trainable params: 234,128 (914.56 KB)

 Non-trainable params: 0 (0.00 B)

None
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 17ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step


((60000, 4), (10000, 4))

In [ ]:
#del extract
del extract
del vqkdc_zzfm_hea_lenet

In [ ]:
import gc

# Call garbage collector
gc.collect()

1013

In [ ]:
## training the VQKDC quantum circuit with features in a generative way
vqkdc_zzfm_hea = VQKDC_MIXED_ZZFM_HEA(dim_x_param = NUM_QUBITS_FFS, n_zzfm_qubits = NUM_QUBITS_FFS, n_ancilla_qubits =  NUM_ANCILLA_QUBITS, num_classes_qubits = NUM_CLASSES_QUBITS, num_classes_param = NUM_CLASSES, n_training_data = N_TRAINING_DATA, num_zzfm_layers = NUM_ZZFM_LAYERS, reduction = REDUCTION, training_type = TRAINING_TYPE, num_layers_hea = NUM_LAYERS_HEA, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE)
vqkdc_zzfm_hea.fit(X_train_feats, y_train_oh, epochs = EPOCHS)

Epoch 1/4
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1333s 175ms/step - accuracy: 0.7137 - loss: 3.1419
Epoch 2/4
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 653s 174ms/step - accuracy: 0.9868 - loss: 2.3519
Epoch 3/4
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 655s 175ms/step - accuracy: 0.9877 - loss: 2.3406
Epoch 4/4
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 685s 176ms/step - accuracy: 0.9873 - loss: 2.3396


In [ ]:
print(vqkdc_zzfm_hea.model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ quantum_layer_1 (QuantumLayer)       │ (16, 10)                    │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,080 (24.06 KB)

 Trainable params: 1,026 (8.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,054 (16.04 KB)

None


In [ ]:
y_pred = vqkdc_zzfm_hea.predict(X_test_feats)

accuracy_score(y_test, np.argmax(y_pred, axis=1))

313/313 ━━━━━━━━━━━━━━━━━━━━ 594s 1s/step


0.9846

In [ ]:
print("a")

a


# Draft code

## Baseline model 2: Mixed QMC variational, ZZFM, with Le-Net Conv layer with HEA, Discriminative then generative learning, additional linear layer from dim of latent space to dim 4.

This model works but break the RAM

The number of parameters of the Hardware efficient ansatz is given by,


```
HEA_size = n_qubits * (num_layers_hea + 1) * 2
```



In [ ]:
### Quantum variational KDC with QEFF

import tensorcircuit as tc
from tensorcircuit import keras
import tensorflow as tf

from functools import partial
import numpy as np
import math as m
from scipy.stats import entropy, spearmanr

tc.set_backend("tensorflow")
tc.set_dtype("complex128")

pi = tf.constant(m.pi)

class VQKDC_MIXED_ZZFM_HEA_LENET:
    r"""
    Defines the ready-to-use Quantum measurement classification (QMC) model implemented
    in TensorCircuit using the TensorFlow/Keras API. Any additional argument in the methods has to be Keras-compliant.

    Args:
        auto_compile: A boolean to autocompile the model using default settings. (Default True).
        var_pure_state_size:
        gamma:

    Returns:
        An instantiated model ready to train with ad-hoc data.

    """
    def __init__(self, n_qeff_qubits, n_ancilla_qubits, num_classes_qubits, num_classes_param, dim_lenet_out_param,  input_dim_param, gamma, n_training_data, num_zzfm_layers, reduction = "none", training_type = "generative", num_layers_hea = 3, batch_size = 16, learning_rate = 0.0005, auto_compile=True):

        self.circuit = None
        self.gamma = gamma
        self.num_layers_hea = num_layers_hea
        self.num_classes = num_classes_param
        self.num_classes_qubits = num_classes_qubits
        self.n_zzfm_qubits = n_qeff_qubits
        self.n_ancilla_qubits = n_ancilla_qubits
        self.n_total_qubits_temp = self.num_classes_qubits + self.n_zzfm_qubits + self.n_ancilla_qubits
        self.n_training_data = n_training_data
        self.dim_lenet_out = dim_lenet_out_param
        self.input_dim = input_dim_param
        self.num_zzfm_layers = num_zzfm_layers
        self.var_hea_ansatz_size = int(self.n_total_qubits_temp*(self.num_layers_hea+1)*2)
        self.reduction  = reduction
        self.training_type = training_type
        self.learning_rate = learning_rate
        self.batch_size = batch_size

        layer = keras.QuantumLayer(
            partial(self.layer),
            [(self.var_hea_ansatz_size,)]
            )

        ## build the Let-net model
        self.model = Sequential() ## keep for old code
        #self.model = Sequential() ## new code
        #self.model.add(tf.keras.layers.InputLayer(input_shape=(self.input_dim, self.input_dim, 1))) ## new code
        self.model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=(5, 5), padding="same", activation='relu', input_shape=(self.input_dim, self.input_dim, 1)))
        self.model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
        self.model.add(tf.keras.layers.Conv2D(filters=50, kernel_size=(5, 5), padding="same", activation='relu'))
        self.model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2))
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(units=84, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=self.dim_lenet_out, activation = None)) # original "softmax"
        self.model.add(tf.keras.layers.Dense(units=self.n_zzfm_qubits, activation = None))  ## we add this additional layer compared to normal models

        # add the quantum layer

        self.model.add(layer)
        print(self.model.summary())

        if auto_compile:
            self.compile()

    def layer(
            self,
            x_sample_param,
            var_hea_ansatz_param,
        ):
        r"""
        Defines a Density Matrix Kernel Density Estimation quantum layer for learning with fixed qaff (Meaning of qaff?). (This function was originally named dmkde_mixed_variational_density_estimation_fixed_qaff)

        Args:
            U_dagger:
            var_pure_state_param:

        Returns:
            The probabilities of :math:`|k\rangle`, `|1\rangle`, ..., `|k\rangle` state for kernel density classification of the classes.
        """

        ### indices pure state hea
        index_iter_hea  = iter(np.arange(len(var_hea_ansatz_param)))

        ### indices classes, of ms
        n_qubits_classes_zzfm_temp = self.num_classes_qubits + self.n_zzfm_qubits
        index_qubit_states = _indices_qubits_classes(n_qubits_classes_zzfm_temp, self.num_classes) # extract indices of the bit string of classes

        # Instantiate a circuit with the calculated number of qubits.
        self.circuit = tc.Circuit(self.n_total_qubits_temp)

        def hea_ansatz(qc_param, num_qubits_param, num_layers_param):
          # encoding
          for i in range (0, num_qubits_param):
            qc_param.ry(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
            qc_param.rz(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
          # layers
          for j in range(num_layers_param):
            for i in range (0, num_qubits_param-1):
              qc_param.CNOT(i, i+1)

            for i in range (0, num_qubits_param):
              qc_param.ry(i, theta= var_hea_ansatz_param[next(index_iter_hea)])
              qc_param.rz(i, theta= var_hea_ansatz_param[next(index_iter_hea)])

        ## learning pure state with HEA
        hea_ansatz(self.circuit, self.n_total_qubits_temp, self.num_layers_hea)

        # Value to predict zzfm
        x_sample_temp = tf.expand_dims(x_sample_param, axis=0)
        init_qubit_qeff_temp = self.num_classes_qubits # qubit at which the qaff mapping starts it starts after the qubits of the classes

        for i in range(self.num_zzfm_layers):
          for k in reversed(range(self.n_zzfm_qubits)):
            for j in reversed(range(0, k)):
              self.circuit.cnot(init_qubit_qeff_temp + j, init_qubit_qeff_temp + k)
              self.circuit.rz(init_qubit_qeff_temp + k, theta = 2*(math.pi-x_sample_temp[0][j])*(math.pi-x_sample_temp[0][k]))
              self.circuit.cnot(init_qubit_qeff_temp + j, init_qubit_qeff_temp + k)

          for l in range(self.n_zzfm_qubits):
            self.circuit.rz(init_qubit_qeff_temp + l, theta = 2*(x_sample_temp[0][l]))

          for m in range(init_qubit_qeff_temp, init_qubit_qeff_temp + self.n_zzfm_qubits):
            self.circuit.H(m)

        # Trace out ancilla qubits, find probability of [000] state for density estimation
        measurement_state = tc.quantum.reduced_density_matrix(
                        self.circuit.state(),
                        cut=[m for m in range(n_qubits_classes_zzfm_temp, self.n_total_qubits_temp)])
        measurements_results = tc.backend.real(tf.stack([measurement_state[index_qubit_states[i], index_qubit_states[i]] for i in range(self.num_classes)]))
        if self.training_type == "discriminative":
          measurements_results = measurements_results / tf.reduce_sum(measurements_results, axis = -1)
        return measurements_results

    def custom_categorical_crossentropy(self, y_true, y_pred):
      ## code generated with the aid of chat gpt
      """
      Compute the categorical cross-entropy loss with mean reduction.

      Args:
      y_true: Tensor of true labels, shape (batch_size, num_classes).
      y_pred: Tensor of predicted probabilities, shape (batch_size, num_classes).

      Returns:
      Scalar tensor representing the mean loss over the batch.
      """
      # Ensure predictions are clipped to avoid log(0)
      epsilon_two = 1e-7  # small constant to avoid division by zero
      y_pred = tf.clip_by_value(y_pred, epsilon_two, np.inf)  # clip values to avoid log(0) originaly 1.0 - epsilon

      # Compute the categorical cross-entropy loss for each sample
      loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)

      if self.reduction == "none":
        return loss
      elif self.reduction == "mean":
        # Compute the mean loss over the batch
        mean_loss = tf.reduce_mean(loss)
        return mean_loss
      elif self.reduction == "sum":
        # Compute the sum loss over the batch
        sum_loss = tf.reduce_sum(loss)
        return sum_loss
      else:
        return loss

    def compile(
            self,
            optimizer=tf.keras.optimizers.Adam,
            **kwargs):
        r"""
        Method to compile the model.

        Args:
            optimizer:
            **kwargs: Any additional argument.

        Returns:
            None.
        """
        self.model.compile(
            loss = self.custom_categorical_crossentropy,
            optimizer=optimizer(self.learning_rate),
            metrics=["accuracy"],
            **kwargs
        )

    def fit(self, x_train, y_train, batch_size=16, epochs = 30, **kwargs):
        r"""
        Method to fit (train) the model using the ad-hoc dataset.

        Args:
            x_train:
            y_train:
            batch_size:
            epochs:
            **kwargs: Any additional argument.

        Returns:
            None.
        """

        self.model.fit(x_train, y_train, batch_size = self.batch_size, epochs = epochs, **kwargs)

    def predict(self, x_test):
      r"""
      Method to make predictions with the trained model.

      Args:
          x_test:

      Returns:
          The predictions of the conditional density estimation of the input data.
      """
      return (tf.experimental.numpy.power((self.gamma/(pi)), self.dim_lenet_out/2.)*\
          self.model.predict(x_test)).numpy()

    def extract_lenet_features(self, x_test):
      r"""
      Method to extract the features of the Lenet.

      Args:
          x_test:

      Returns:
          Features of the Lenet.
      """
      extract = Model(self.model.inputs, self.model.layers[-2].output)
      new_model = Sequential()
      new_model.add(extract)
      return new_model.predict(x_test)

    def freeze_lenet(self):
      r"""
      Method to frezze the layers of the Lenet.

      Args:
          self:

      Returns:
          None.
      """
      for layer in self.model.layers[:-1]:
          layer.trainable = False

    def set_training_type(self, new_training_type):
      r"""
      Method to change the training type of the method during training.

      Args:
          new_training_strategy:

      Returns:
          None.
      """
      self.training_type = new_training_type

In [ ]:
## training the quantum circuit
# Define a LeNet CNN feature extraction model
input_shape = X_train.shape[1:]
INPUT_DIM = input_shape[0]
NUM_QUBITS_FFS = 4 ## originally N_QEFFS = 16
NUM_CLASSES = 10
NUM_CLASSES_QUBITS = 4
DIM_LENET_OUT = 30 # originally 16
GAMMA = 2**(-2) ## originally 2**(0)
EPOCHS = 4
N_TRAINING_DATA = X_train.shape[0]
BATCH_SIZE = 16
NUM_ZZFM_LAYERS = 2 ## set 2 for final experiments
LEARNING_RATE = 0.0005 ## originally 0.0005
RANDOM_STATE_QEFF = 123
NUM_ANCILLA_QUBITS = 1
NUM_TOTAL_QUBITS = NUM_QUBITS_FFS + NUM_ANCILLA_QUBITS + NUM_CLASSES_QUBITS
NUM_LAYERS_HEA = int(np.round(((2**NUM_TOTAL_QUBITS-1)/NUM_TOTAL_QUBITS)-1))
TRAINING_TYPE = "generative"
REDUCTION = "none"

In [ ]:
## this code creates a discriminative model
vqkdc_zzfm_hea = VQKDC_MIXED_ZZFM_HEA_LENET(n_qeff_qubits = NUM_QUBITS_FFS, n_ancilla_qubits =  NUM_ANCILLA_QUBITS, num_classes_qubits = NUM_CLASSES_QUBITS, num_classes_param = NUM_CLASSES, dim_lenet_out_param = DIM_LENET_OUT, input_dim_param = INPUT_DIM, gamma=GAMMA, n_training_data = N_TRAINING_DATA, num_zzfm_layers = NUM_ZZFM_LAYERS, reduction = REDUCTION, training_type = TRAINING_TYPE, num_layers_hea = NUM_LAYERS_HEA, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, auto_compile = False)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
vqkdc_zzfm_hea.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
vqkdc_zzfm_hea.fit(X_train, y_train_oh, epochs = EPOCHS)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 20)          │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 14, 14, 20)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 14, 14, 50)          │          25,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_1                  │ (None, 7, 7, 50)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2450)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 84)                  │         205,884 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │             124 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ quantum_layer (QuantumLayer)         │ (None, 10)                  │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 235,154 (922.58 KB)

 Trainable params: 235,154 (922.58 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/4
 258/3750 ━━━━━━━━━━━━━━━━━━━━ 11:13 193ms/step - accuracy: 0.2316 - loss: 2.9940

KeyboardInterrupt: 

In [ ]:
## training the quantum circuit
vqkdc_zzfm_hea_generative = VQKDC_MIXED_ZZFM_HEA(dim_x_param = NUM_QUBITS_FFS, n_qeff_qubits = NUM_QUBITS_FFS, n_ancilla_qubits =  NUM_ANCILLA_QUBITS, num_classes_qubits = NUM_CLASSES_QUBITS, num_classes_param = NUM_CLASSES, n_training_data = N_TRAINING_DATA, num_zzfm_layers = NUM_ZZFM_LAYERS, reduction = REDUCTION, training_type = TRAINING_TYPE, num_layers_hea = NUM_LAYERS_HEA, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE)

vqkdc_zzfm_hea_generative.fit(X_train_feats, y_train_oh, epochs = EPOCHS)

In [ ]:
# this code frezzes the weights of the Le net layer, and then sets the model to be trained in a generative way
for layer in vqkdc_zzfm_hea.model.layers[:-1]:
    layer.trainable = False
vqkdc_zzfm_hea.compile()
vqkdc_zzfm_hea.fit(X_train, y_train_oh, epochs = EPOCHS)

NameError: name 'vqkdc_zzfm_hea' is not defined

In [ ]:
y_pred = vqkdc_zzfm_hea.predict(X_test)

accuracy_score(y_test, np.argmax(y_pred, axis=1))